<a href="https://colab.research.google.com/github/crebi6/GIS/blob/main/UsaPopulationDashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the the population data

In [2]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/JoshData/historical-state-population-csv/refs/heads/primary/historical_state_population_by_year.csv", names = ['state', 'year', 'population'])
df

,state,year,population
0,AK,1950,135000
1,AK,1951,158000
2,AK,1952,189000
3,AK,1953,205000
4,AK,1954,215000
...,...,...,...
6270,WY,2020,577681
6271,WY,2021,579636
6272,WY,2022,581978
6273,WY,2023,585067


# **make the dashboard**

***before that download necessary libraries***

Over the past century, the United States has undergone dramatic changes. From the bustling cities of the East Coast to the sprawling suburbs of the West, population growth has reshaped the nation. But not all states have grown at the same pace. Some have exploded in size, while others have grown more slowly. Today, we’ll explore the story of how America’s population has evolved—and what it means for the future. Interact with the map and enjoy.

I am Sam, a GIS enthusiast and a full-stack developer, feel free to reach out for projects and more exciting things.

email: samuelmusyoka086@gmail.com
whatsapp: +254757556067

In [6]:
!pip install pandas
!pip install plotly
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [7]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output



# Preprocessing: Ensure state abbreviations are uppercase
df['state'] = df['state'].str.upper()

# Create Dash app
app = Dash(__name__)

# App layout with a dropdown for year selection
app.layout = html.Div([
    html.H1("US State Population Dashboard",
            style={'textAlign': 'center', 'marginBottom': '20px'}),

    # Dropdown to select the year
    html.Div([
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in sorted(df['year'].unique())],
            value=df['year'].max(),  # Default to the most recent year
            clearable=False,
            style={'width': '200px'}
        )
    ], style={'textAlign': 'center', 'marginBottom': '20px'}),

    # Graph components for the map and the trend
    dcc.Graph(id='state-population-map'),
    dcc.Graph(id='state-population-trend')
])

# Callback to update the choropleth map based on the selected year
@app.callback(
    Output('state-population-map', 'figure'),
    Input('year-dropdown', 'value')
)
def update_map(selected_year):
    filtered_data = df[df['year'] == selected_year]
    fig = px.choropleth(
        filtered_data,
        locations='state',
        locationmode="USA-states",
        color='population',
        scope="usa",
        color_continuous_scale="Viridis",
        title=f'US State Populations in {selected_year}'
    )
    fig.update_layout(
        geo_scope='usa',
        height=600
    )
    return fig

# Callback to update the trend graph based on clicking a state in the map
@app.callback(
    Output('state-population-trend', 'figure'),
    Input('state-population-map', 'clickData')
)
def update_trend(clickData):
    if clickData:
        # Extract the state abbreviation from the clicked point
        selected_state = clickData['points'][0]['location']
    else:
        # Default to the first state if no click has occurred
        selected_state = df['state'].unique()[0]

    state_data = df[df['state'] == selected_state]
    fig = px.line(
        state_data,
        x='year',
        y='population',
        title=f'Population Trend for {selected_state}'
    )
    fig.update_layout(
        height=400,
        title_x=0.5
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>